### Импортим либы

In [230]:
import random
import math
from typing import Callable
import tabulate

### Делаем два генератора случайных величин с равномерным и экспоненциальным распределением

Вызывать так:
``` python
obj = even_int_gen(3, 5)
next(obj)
# 3, или 4, или даже 5...
# Жеееесть...
```

In [259]:
def even_int_gen(a: int, b: int) -> int:
    # чтобы округление не уменьшало количество генераций крайних чисел вдвое
    a -= 0.5 
    b += 0.5
    while True:
        yield round(a + random.random() * (b - a))

# тут округление тоже возможно что-то меняет, но мне все равно
def exp_int_gen(l: float) -> int:
    while True:
        yield (-1) * math.log(random.random()) / l

### Напишем функцию, делающую аналитический расчет вероятности для системы СМО Эрланга

In [232]:
def erlang_model(l: float, u: float, stands_count: int) -> float:
    alpha = l / u
    n = stands_count
    up = alpha ** n / math.factorial(n)
    down = 1
    for i in range(1, n + 1):
        down += alpha ** i / math.factorial(i)
    return 1 - up / down

### Тестим

In [248]:
l = 0.95
u = 1

table = []
for chan in range(1, 8):
    table.append([chan, erlang_model(l, u, chan)])

print("λ =", l)
print("μ =", u)

headers = ["n_k", "p_priem"]
print(tabulate.tabulate(table, headers=headers, tablefmt="fancy_grid", floatfmt=".5f"))

λ = 0.95
μ = 1
╒═══════╤═══════════╕
│   n_k │   p_priem │
╞═══════╪═══════════╡
│     1 │   0.51282 │
├───────┼───────────┤
│     2 │   0.81208 │
├───────┼───────────┤
│     3 │   0.94383 │
├───────┼───────────┤
│     4 │   0.98684 │
├───────┼───────────┤
│     5 │   0.99751 │
├───────┼───────────┤
│     6 │   0.99961 │
├───────┼───────────┤
│     7 │   0.99995 │
╘═══════╧═══════════╛


### Напишем функцию имитационного моделирования

На вход идет:
1. **items_count** - количество деталей для обработки
2. **stands_count** - количество каналов (стендов) для обработки
3. **release_gen** - генератор значений времени входного канала
4. **check_gen** - генератор значений времени обработки на стенде

Засчет того, что мы даем функции на вход генераторы, то можно дать как генератор равномерно распределенных чисел, так и генератор экспоненциально распределенных чисел.

Это позволит легко, без переписывания функции в дальнейшем, отладить ее работу, сравнивая значения с СМО Эрланга, которая использует экспоненциальное распределение, а нам в дальнейшем понадобится равномерное.

In [234]:
# тут аннотации неоч верные, ибо yield требует SupprotsNext, а я пишу Callable
def imit_model(items_count: int, stands_count: int,
               release_gen: Callable[[int, int], int],
               check_gen: Callable[[int, int], int]) -> float:

    stands = [0 for _ in range(stands_count)]
    not_checked = 0

    for i in range(items_count):
        rel_time = next(release_gen)

        for j in range(len(stands)):
            stands[j] = max(0, stands[j] - rel_time)

        found_free = False
        for j in range(len(stands)):
            if stands[j] == 0:
                stands[j] = next(check_gen)
                found_free = True
                break

        if not found_free:
            not_checked += 1

    return (items_count - not_checked) / items_count

### Тестим

In [250]:
l = 0.95
u = 1
release_gen = exp_int_gen(l)
check_gen = exp_int_gen(u)

need = 0.98
items_count = 10**5
stands_count = 1

table = []
while True:
    result = imit_model(items_count, stands_count, release_gen, check_gen)
    erl_result = erlang_model(l, u, stands_count)
    table.append([stands_count, result, erl_result])
    stands_count += 1
    if result >= need:
        break

print("λ =", l)
print("μ =", u)
print("Количество изделий на входе:", items_count)
print("Контроль не менее", str(need * 100), "% изделий")

headers = ["n_k", "имитация", "Эрланг"]
print(tabulate.tabulate(table, headers=headers, tablefmt="fancy_grid", floatfmt=".5f"))

λ = 0.95
μ = 1
Количество изделий на входе: 100000
Контроль не менее 98.0 % изделий
╒═══════╤════════════╤══════════╕
│   n_k │   имитация │   Эрланг │
╞═══════╪════════════╪══════════╡
│     1 │    0.51599 │  0.51282 │
├───────┼────────────┼──────────┤
│     2 │    0.81310 │  0.81208 │
├───────┼────────────┼──────────┤
│     3 │    0.94273 │  0.94383 │
├───────┼────────────┼──────────┤
│     4 │    0.98667 │  0.98684 │
╘═══════╧════════════╧══════════╛


Как видно, имитация показывает верный результат относительно аналитического решения СМО Эрланга

### Проверим систему с равномерным распределением

In [270]:
a, b = 5, 15
c, d = 15, 25
release_gen = even_int_gen(a, b)
check_gen = even_int_gen(c, d)

need = 0.98
items_count = 10**5
stands_count = 1

table = []
while True:
    result = imit_model(items_count, stands_count, release_gen, check_gen)
    table.append([stands_count, result])
    stands_count += 1
    if result >= need:
        break

print("a, b =", [a, b])
print("c, d =", [c, d])
print("Количество изделий на входе:", items_count)
print("Контроль не менее", str(need * 100), "% изделий")

headers = ["n_k", "p_priema"]
print(tabulate.tabulate(table, headers=headers, tablefmt="fancy_grid", floatfmt=".5f"))

a, b = [5, 15]
c, d = [15, 25]
Количество изделий на входе: 100000
Контроль не менее 98.0 % изделий
╒═══════╤════════════╕
│   n_k │   p_priema │
╞═══════╪════════════╡
│     1 │    0.39900 │
├───────┼────────────┤
│     2 │    0.75510 │
├───────┼────────────┤
│     3 │    0.96651 │
├───────┼────────────┤
│     4 │    0.99950 │
╘═══════╧════════════╛
